<a href="https://colab.research.google.com/github/Surya-prakash-v/MLAI/blob/master/kerasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Analytics Vidya</b>
###Face detection and counting</p></b>
<p>People detection and head counting is one of the classical albeit challenging computer vision application. For this problem, given a group selfie/photo, you are required to count the number of heads present in the picture. You are provided with a training set of images with coordinates of bounding box and head count for each image and need to predict the headcount for each image in the test set.</p></br>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import zipfile as zip
import os

zip_ref = zip.ZipFile('./drive/My Drive/Colab Notebooks/AnalyticsVidya/train_HNzkrPW (1).zip', 'r')
zip_ref.extractall('train/')
zip_ref.close()

In [51]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

IMG_SIZE = 128
BATCH_SIZE = 32
traindatagen = ImageDataGenerator(rescale=1./255)

traincsv = pd.read_csv('train/train.csv',dtype={'Name': str,'HeadCount':int})
trainFaceMaskcsv = pd.read_csv('train/bbox_train.csv',dtype={'Name': str,'width':float,'height':float,'xmin':float,'ymin':float,'xmax':float,'ymax':float})
trainFaceMaskcsv['xmin'] = trainFaceMaskcsv['xmin']*IMG_SIZE/trainFaceMaskcsv['width']
trainFaceMaskcsv['xmax'] = trainFaceMaskcsv['xmax']*IMG_SIZE/trainFaceMaskcsv['width']
trainFaceMaskcsv['ymin'] = trainFaceMaskcsv['ymin']*IMG_SIZE/trainFaceMaskcsv['height']
trainFaceMaskcsv['ymax'] = trainFaceMaskcsv['ymax']*IMG_SIZE/trainFaceMaskcsv['height']

traingenerator = traindatagen.flow_from_dataframe(traincsv, directory='train/image_data/',x_col='Name', y_col=['Name','HeadCount'], 
                    target_size=(IMG_SIZE,IMG_SIZE), color_mode='rgb',class_mode='raw', batch_size=32,shuffle=False, seed=10)

Found 5733 validated image filenames.


####Helper functions</b>

In [52]:
def get_faceMasks(batch_y,anchor):
  facemasks = np.zeros(shape=(batch_y.shape[0],int(IMG_SIZE/anchor),int(IMG_SIZE/anchor),6))
  #i = 0
  #for element in y_train_dataset:
  for i in range(batch_y.shape[0]):
    faceList = trainFaceMaskcsv.where(trainFaceMaskcsv['Name']==batch_y[i][0])
    faceList = faceList[faceList.Name.notnull()]
    arr = []
    for face in range(faceList['Name'].count()):
      for row_x in range(0,IMG_SIZE,anchor):
        for col_y in range(0,IMG_SIZE,anchor): 
          arr.append([row_x,col_y,row_x+anchor,col_y+anchor,faceList['xmin'].iloc[face],faceList['ymin'].iloc[face],faceList['xmax'].iloc[face],faceList['ymax'].iloc[face],0,0,0,0,0,faceList['Name'].count()])
    ious = get_iou(np.array(arr))
    ious2 = pd.DataFrame(ious)
    idx = ious2.loc[ious2.reset_index().groupby([0,1,2,3])[12].idxmax()]
    facemasks[i] = np.reshape((np.array(idx)/IMG_SIZE)[:,[8,9,10,11,12,13]],newshape=(int(IMG_SIZE/anchor),int(IMG_SIZE/anchor),6))
    #i = i+1
  #return tf.data.Dataset.from_tensor_slices(facemasks)
  return facemasks

In [53]:
import numpy as np
def get_iou(a, epsilon=1e-5):
    a[:,8] = np.maximum(a[:,0], a[:,4])
    a[:,9] = np.maximum(a[:,1], a[:,5])
    a[:,10] = np.minimum(a[:,2], a[:,6])
    a[:,11] = np.minimum(a[:,3], a[:,7])
    overlap = np.multiply(np.maximum(0,np.subtract(a[:,10],a[:,8])),np.maximum(0,np.subtract(a[:,11],a[:,9])))
    area_a = np.multiply(np.subtract(a[:,2],a[:,0]),np.subtract(a[:,3],a[:,1]))
    area_b = np.multiply(np.subtract(a[:,6],a[:,4]),np.subtract(a[:,7],a[:,5]))
    area_combined = np.subtract(area_a+area_b,overlap)+epsilon
    a[:,12] = np.divide(overlap,area_combined)*IMG_SIZE
    return a

In [54]:
import numpy as np
import tensorflow as tf
def get_output(batch):
  while True:
    batch_x, batch_y = next(batch)
    #batch_x_dataset = tf.data.Dataset.from_tensor_slices(batch_x)
    #batch_y_dataset = tf.data.Dataset.from_tensor_slices(batch_y)
    yield (batch_x, [get_faceMasks(batch_y,8),get_faceMasks(batch_y,16),get_faceMasks(batch_y,32),get_faceMasks(batch_y,64)])
    #yield tf.data.Dataset.from_tensor_slices((batch_x_dataset, [get_faceMasks(batch_y_dataset,8),get_faceMasks(batch_y_dataset,16),get_faceMasks(batch_y_dataset,32),get_faceMasks(batch_y_dataset,64)]))

####Test Pre</b>

In [55]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

x_train,y_train = next(get_output(traingenerator))
for pic in range(7):
  im = np.array(x_train[pic*4])
  fig,ax = plt.subplots(pic,figsize = (8, 8))
  ax.imshow(im)
  for anchor in range(len(y_train)):
    boxes = np.reshape(y_train[30][anchor],newshape=(int((IMG_SIZE*IMG_SIZE)/(anchor_sizes[anchor]*anchor_sizes[anchor])),13))
    boxes = boxes[boxes[:,12]>0]*IMG_SIZE
    for box in range(len(boxes)):
      ax.add_patch(patches.Rectangle((boxes[box,8],boxes[box,9]),boxes[box,10]-boxes[box,8],boxes[box,11]-boxes[box,9],linewidth=1,edgecolor='r',facecolor='none'))
plt.show()

AttributeError: ignored

<Figure size 576x576 with 0 Axes>

In [56]:
import tensorflow as tf
from tensorflow.python.keras.layers import (Flatten,LeakyReLU,AveragePooling2D,concatenate,
                                            Reshape,Activation,Input, Conv2D, MaxPooling2D,
                                            BatchNormalization, GlobalAveragePooling2D,SpatialDropout2D)
def resNetBlock(input_image,num):

  layer1 = Conv2D(128, 3, strides=(1,1), padding='same', use_bias=False,name ='rone'+str(num))(input_image)
  layer1 = BatchNormalization()(layer1)
  layer1 = SpatialDropout2D(0.25)(layer1)
  layer1 = LeakyReLU(alpha=0.1)(layer1)
  
  layer3 = Conv2D(64, 1, strides=(1,1), padding='same', use_bias=False,name='rtwo'+str(num))(layer1)
  layer3 = BatchNormalization()(layer3)
  layer3 = SpatialDropout2D(0.25)(layer3)
  layer3 = LeakyReLU(alpha=0.1)(layer3)
    
  return layer3  

def build_model(img_size,img_ch):
  input_sh = Input(shape=(img_size,img_size,img_ch)) 

  layer_1 = Conv2D(16, 3, strides=(1,1), padding='same', use_bias=False,name='one')(input_sh)
  layer_1 = BatchNormalization()(layer_1)
  layer_1 = SpatialDropout2D(0.25)(layer_1)
  layer_1 = LeakyReLU(alpha=0.1)(layer_1)
  
  layer_2 = Conv2D(32, 3, strides=(1,1), padding='same', use_bias=False,name='two')(layer_1)
  layer_2 = BatchNormalization()(layer_2)
  layer_2 = SpatialDropout2D(0.25)(layer_2)
  layer_2 = LeakyReLU(alpha=0.1)(layer_2)
  
  layer_3 = Conv2D(64, 3, strides=(1,1), padding='same', use_bias=False,name='three')(layer_2)
  layer_3 = BatchNormalization()(layer_3)
  layer_3 = SpatialDropout2D(0.25)(layer_3)
  layer_3 = LeakyReLU(alpha=0.1)(layer_3)
 
  resNetBlock1 = resNetBlock(layer_3,1)
  concact1 = concatenate([resNetBlock1, layer_3])
  layer_l = LeakyReLU(alpha=0.1)(concact1)
  maxpool1 = MaxPooling2D(pool_size=(2, 2))(layer_l)

  resNetBlock2 = resNetBlock(maxpool1,2)
  outputy = Conv2D(8, 1, strides=(1,1), padding='same', use_bias=False,name='four')(resNetBlock2)
  concact2 = concatenate([resNetBlock2, maxpool1])
  layer_l2 = LeakyReLU(alpha=0.1)(concact2)
  maxpool2 = MaxPooling2D(pool_size=(2, 2))(layer_l2)

  resNetBlock3 = resNetBlock(maxpool2,3)
  outputz = Conv2D(16, 1, strides=(1,1), padding='same', use_bias=False,name='five')(resNetBlock3)
  concact3 = concatenate([resNetBlock3, maxpool2])
  layer_l3 = LeakyReLU(alpha=0.1)(concact3)
  maxpool3 = MaxPooling2D(pool_size=(2, 2))(layer_l3)
  
  resNetBlock4 = resNetBlock(maxpool3,4)
  outputw = Conv2D(32, 1, strides=(1,1), padding='same', use_bias=False,name='six')(resNetBlock4)
  concact4 = concatenate([resNetBlock4, maxpool3])
  layer_l4 = LeakyReLU(alpha=0.1)(concact4)
  maxpool4 = MaxPooling2D(pool_size=(2, 2))(layer_l4)
  
  resNetBlock5 = resNetBlock(maxpool4,5)
  outputk = Conv2D(6, 1, strides=(1,1), padding='same', use_bias=False,name='seven')(resNetBlock5)
  concact5 = concatenate([resNetBlock5, maxpool4])
  layer_l5 = LeakyReLU(alpha=0.1)(concact5)
  maxpool5 = MaxPooling2D(pool_size=(2, 2))(layer_l5)
  
  resNetBlock6 = resNetBlock(maxpool5,6)
  concact6 = concatenate([resNetBlock6, maxpool5])
  layer_l6 = LeakyReLU(alpha=0.1)(concact6)

  layer_ant = Conv2D(6, (1,1), strides=(1,1), padding='same', use_bias=False, name='eight')(layer_l6)
  layer_plus1 = Conv2D(6, (3,3), strides=(1,1),  use_bias=False, name='nine')(layer_ant)
  layer_plus2 = Conv2D(6, (3,3), strides=(1,1),padding='same', use_bias=False, name='ten')(layer_plus1)
  #layer_plus3 = Conv2D(6, (3,3), strides=(1,1),  use_bias=False, name='eleven')(layer_plus2)
  #model = Model(input_sh,outputs=[outputw,outputk,layer_ant,layer_plus2,layer_plus3])
  model = tf.keras.Model(input_sh,outputs=[outputw,outputk,layer_ant,layer_plus2])
  return model

In [61]:
import tensorflow_addons as tfa
def my_loss5(y_true,y_pred):
  loss_classification = tf.math.abs(tf.math.subtract(y_true[:,:,:,4],y_pred[:,:,:,4]))
  loss_regression = tf.math.abs(tf.math.subtract(y_true[:,:,:,5],y_pred[:,:,:,5]))
  gl = tfa.losses.GIoULoss()
  loss = tf.keras.losses.Reduction.SUM(loss_classification)+tf.keras.losses.Reduction.SUM(loss_regression)+tf.keras.losses.Reduction.SUM(gl(y_true[:,:,:,0:4], y_pred[:,:,:,0:4]))
  return loss

In [58]:
current_model= build_model(None,3)
current_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
one (Conv2D)                    (None, None, None, 1 432         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, None, None, 1 64          one[0][0]                        
__________________________________________________________________________________________________
spatial_dropout2d_15 (SpatialDr (None, None, None, 1 0           batch_normalization_15[0][0]     
____________________________________________________________________________________________

In [59]:
from tensorflow.python.keras.callbacks import ModelCheckpoint
def learning_strategy(image_size,previous_epoch,epoch_size,batchsize,learning_rate):
  
  tf.keras.backend.clear_session()

  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  print("All devices: ", tf.config.list_logical_devices('TPU'))
  strategy = tf.distribute.TPUStrategy(resolver)

  with strategy.scope():
    modelx = build_model(image_size,3)
  #current_model.compile(loss={'six':my_loss5,'seven':my_loss5,'eight':my_loss5,'ten':my_loss5,'eleven':my_loss5} ,optimizer='adam',metrics=['accuracy'])
    modelx.compile(loss={'six':my_loss5,'seven':my_loss5,'eight':my_loss5,'ten':my_loss5} ,optimizer='adam',metrics=['accuracy'])
    #modelx.compile(loss='mape' ,optimizer='adam',metrics=['accuracy'])
  
    #checkpoint = ModelCheckpoint('drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection/Weights/face_detectionTPU.hdf5',monitor='ten_accuracy',
     #                        save_weights_only=False,save_best_only = True,mode='auto')
  #checkpoint = ModelCheckpoint('/gdrive/My Drive/tpuweights/vddddAssignment4Amodel_{epoch:02d}_{val_loss:.2f}.hdf5',monitor='val_acc',save_weights_only=True,save_best_only = True,mode='auto')
    #modelx.fit(get_output(traingenerator),initial_epoch=previous_epoch,epochs=epoch_size,steps_per_epoch=int(5733/32),verbose=1,callbacks=[checkpoint])   
    modelx.fit(get_output(traingenerator),initial_epoch=previous_epoch,epochs=epoch_size,steps_per_epoch=int(5733/32),verbose=1)                       

In [62]:
learning_strategy(128,0,20,32,0.001)

INFO:tensorflow:Initializing the TPU system: grpc://10.28.178.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.178.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/20


ValueError: ignored

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

IMAGE_SIZE = [128, 128]

batch_size = 16 * tpu_strategy.num_replicas_in_sync

gcs_pattern = 'gs://faces/tfrecords-jpeg-128/*.tfrec'
filenames = tf.io.gfile.glob(gcs_pattern)
train_fns = filenames[:split]

def parse_tfrecord(example):
  features = {
    "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
    "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar
    "one_hot_class": tf.io.VarLenFeature(tf.float32),
  }
  example = tf.io.parse_single_example(example, features)
  decoded = tf.image.decode_jpeg(example['image'], channels=3)
  normalized = tf.cast(decoded, tf.float32) / 255.0 # convert each 0-255 value to floats in [0, 1] range
  image_tensor = tf.reshape(normalized, [*IMAGE_SIZE, 3])
  one_hot_class = tf.reshape(tf.sparse.to_dense(example['one_hot_class']), [5])
  return image_tensor, one_hot_class

In [ ]:
from tensorflow.python.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection/Weights/face_detectionTPU.hdf5',monitor='ten_accuracy',
                             save_weights_only=False,save_best_only = True,mode='auto')
current_model.fit(get_output(traingenerator),epochs=50,verbose=1,steps_per_epoch=int(5733/32),callbacks=[checkpoint])

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image,ImageOps,ImageFilter
import matplotlib.patches as patches
import random
import cv2
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier('haarcascade_profileface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
item_num = random.randint(0,(x_ou[0].shape[0])-1)
#item_num = 24
img = Image.fromarray((x_ou[0][item_num]).astype('uint8'),mode='RGB')
print(item_num)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ticks = np.arange(0, 256, 32)

ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.grid(b=True,color='black')
gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.01, 10)
for (x1,y1,w1,h1) in faces:
    print('x1 '+str(x1)+' y1 '+str(y1)+' w1 '+str(w1)+' h1 '+str(h1))
    img = cv2.rectangle(np.array(img),(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
    roi_color = img[y1:y1+h1, x1:x1+w1]

profiles = profile_cascade.detectMultiScale(gray, 1.01, 10)
for (x2,y2,w2,h2) in profiles:
    print('x2 '+str(x2)+' y2 '+str(y2)+' w2 '+str(w2)+' h2 '+str(h2))
    img = cv2.rectangle(np.array(img),(x2,y2),(x2+w2,y2+h2),(0,255,0),2)
    roi_color = img[y2:y2+h2, x2:x2+w2]

ax.imshow(img)

####Model</b>

16 x 16 x 5 x 2

In [ ]:
import requests
from io import BytesIO
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt


response = requests.get('https://www.bmw.co.uk/bmw-cars/explore-the-range/lifestyle/image-thumb__14913__480x270/3-series-1920x1080.jpeg?1539010591')
img = Image.open(BytesIO(response.content))
img2 = img.resize((32, 32), Image.NEAREST) 
img3 =np.expand_dims(img2, axis=0)
print(img3.shape)
p = current_model.predict(img3)
print(p.shape)
print(p[0])

In [ ]:
def fetch_model(input_size):
  files = os.listdir('/drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection')

  matchs = [s for s in files if SAVED_MODEL in s]
  
  least_val_loss = 1000.00
  previous_model_file = ''
  for m in matchs:
      if float((m.split('_',2)[2]).split('.hdf5',1)[0]) < least_val_loss:
        least_val_loss = float((m.split('_',2)[2]).split('.hdf5',1)[0])
        previous_model_file = '/drive/My Drive/Colab Notebooks/AnalyticsVidya/FaceDetection' + m
  if(least_val_loss < 1000.00):
      print("Loading previous model with least val loss: "+ str(previous_model_file))
      current_model = build_model(img_size=input_size)
      current_model.load_weights(previous_model_file)
  else:
      print("No previous model..   creating new Model")
      current_model = build_model(img_size=input_size)
  
  return current_model